# Experiment 1 Baseline

Checking whether an array is a heap using an RNN

In [1]:
%cd ..
%pwd
import wandb
from experiments.heaps import *
from experiments.utils import *
from baselines.baseline_rnn import *
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

/home/tekne/Oxford/ATML/Project/ggs-nn-model


In [ ]:
def run_model(config, logger=wandb):
    NUM_CLASSES = 2

    model = GraphLevelGGNN(
        annotation_size=1,
        num_layers=config["num_layers"],
        gate_nn=nn.Linear(2 * 1 + config["hidden_size"], 1),
        hidden_size=config["hidden_size"],
        final_layer=nn.Linear(2 * 1 + config["hidden_size"], NUM_CLASSES),
    ).cuda()

    if logger is not None:
        logger.watch(model)
        logger.run.summary["model_parameters"] = model.count_parameters()

    opt = torch.optim.Adam(model.parameters(), lr=config["lr"])
    loss = torch.nn.CrossEntropyLoss()

    training_data, testing_data = get_data(config)

    results = train(
        model=model,
        opt=opt,
        training_data=training_data,
        testing_data=testing_data,
        criterion=lambda out, y: loss(out, y),
        checker=lambda out, y: ((torch.argmax(out, dim=-1) == y).sum(), y.shape[0]),
        epochs=config["epochs"],
        logger=logger
    )

    plt.plot(results["train_loss"], label="Training loss")
    plt.plot(results["test_loss"], label="Testing loss")
    plt.legend()
    plt.show()

    plt.plot(results["train_accuracy"], label="Training accuracy")
    plt.plot(results["test_accuracy"], label="Testing accuracy")
    plt.legend()
    plt.show()
